In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import joblib
emb_matrix_train = joblib.load("/content/drive/MyDrive/SI_AI_KBS/integrazione_AlBERTo_WMAL/sentipolc_agritrend/support_files/sentipol_embedd_7410_128_768_alberto_training.joblib")
wmal_matrix_train = joblib.load("/content/drive/MyDrive/SI_AI_KBS/integrazione_AlBERTo_WMAL/sentipolc_agritrend/support_files/sentipolc_wmal_training_7410_128.joblib")

In [3]:
import pandas as pd
dataset_DF = pd.read_csv("/content/drive/MyDrive/SI_AI_KBS/integrazione_AlBERTo_WMAL/sentipolc_agritrend/data/sentipolc_train_processed.csv")

In [4]:
from sklearn.preprocessing import OneHotEncoder
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
# define example
data = dataset_DF["oneg"]
values = array(data)
print(values)
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
print(integer_encoded)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)
# invert first example
inverted = label_encoder.inverse_transform([argmax(onehot_encoded[0, :])])
print(inverted)

[1 1 1 ... 1 0 0]
[1 1 1 ... 1 0 0]
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [1. 0.]
 [1. 0.]]
[1]


In [5]:
from keras.layers import Input, Dense, LSTM, Bidirectional, TimeDistributed, Flatten, Concatenate
!pip install keras-self-attention

In [6]:
from keras_self_attention import SeqWeightedAttention

In [7]:
import keras
input_emb = Input(shape=(128,768))
input_wmal = Input(shape=(128,))

#Bi-LSTM
bi = Bidirectional(LSTM(32, return_sequences=True))(input_emb)
att = SeqWeightedAttention()(bi)
flat = Flatten()(att)

#WMAL 
d1 = Dense(64) (input_wmal)

#Concat
concat = Concatenate(axis=1)([flat, d1])

d2 = Dense(32) (concat)
output = Dense(2,activation="softmax") (d2)

m = keras.Model(inputs=[input_emb,input_wmal], outputs=output)
m.compile(loss="categorical_crossentropy",optimizer='adam',metrics=["accuracy"])





In [8]:
m.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 768)]   0                                            
__________________________________________________________________________________________________
bidirectional (Bidirectional)   (None, 128, 64)      205056      input_1[0][0]                    
__________________________________________________________________________________________________
seq_weighted_attention (SeqWeig (None, 64)           65          bidirectional[0][0]              
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 128)]        0                                            
_______________________________________________________________________________________

In [9]:
m.fit([emb_matrix_train,wmal_matrix_train],onehot_encoded,batch_size=128, epochs=8)

Epoch 1/8
58/58 [==============================] - 29s 495ms/step - loss: 0.5499 - accuracy: 0.7112
Epoch 2/8
58/58 [==============================] - 29s 493ms/step - loss: 0.4493 - accuracy: 0.7854
Epoch 3/8
58/58 [==============================] - 28s 484ms/step - loss: 0.3924 - accuracy: 0.8165
Epoch 4/8
58/58 [==============================] - 28s 488ms/step - loss: 0.3204 - accuracy: 0.8592
Epoch 5/8
58/58 [==============================] - 28s 486ms/step - loss: 0.2223 - accuracy: 0.9082
Epoch 6/8
58/58 [==============================] - 28s 481ms/step - loss: 0.1334 - accuracy: 0.9499
Epoch 7/8
58/58 [==============================] - 28s 482ms/step - loss: 0.0721 - accuracy: 0.9760
Epoch 8/8
58/58 [==============================] - 28s 490ms/step - loss: 0.0382 - accuracy: 0.9874


In [10]:
emb_matrix_test = joblib.load("/content/drive/MyDrive/SI_AI_KBS/integrazione_AlBERTo_WMAL/sentipolc_agritrend/support_files/creagritrend_embedd_986_128_768_alberto.joblib")
dataset_test = pd.read_csv("/content/drive/MyDrive/SI_AI_KBS/integrazione_AlBERTo_WMAL/sentipolc_agritrend/data/creagritrend_processed.csv")
wmal_matrix_test = joblib.load("/content/drive/MyDrive/SI_AI_KBS/integrazione_AlBERTo_WMAL/sentipolc_agritrend/support_files/creagritrend_wmal_agritrend_986_128.joblib")

In [11]:
# define example
data = dataset_test["label_neg"]
values = array(data)
print(values)
# integer encode
integer_encoded_test = label_encoder.transform(values)
print(integer_encoded_test)
# binary encode
integer_encoded_test = integer_encoded_test.reshape(len(integer_encoded_test), 1)
onehot_encoded_test = onehot_encoder.transform(integer_encoded_test)
print(onehot_encoded_test)

[0 1 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 1 0 0
 0 1 1 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 0 0 1 0 1 1 0 0 0 1 1 0 0
 0 0 0 0 0 1 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 1 0 1 0 1 1 1 1 0 1 1 1 0 0 1 1 0 0
 0 1 1 1 1 1 0 0 0 1 1 1 1 1 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 1 1 0 0 1 0 0
 0 1 0 1 0 1 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0
 0 1 0 0 1 0 1 0 1 0 0 0 0 0 0 0 0 1 0 1 0 1 0 1 1 0 0 0 0 1 0 0 0 0 0 0 1
 1 1 0 1 0 0 1 1 0 1 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0
 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 0 1 1 0 0 0 0 0 0
 0 0 1 0 0 1 0 0 0 1 1 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 1
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 1 0 0 0 0 1 1 0 0 0 1 1 0 0 0 1 1 1 0 1 0
 0 0 0 0 1 1 1 0 0 1 0 0 1 0 1 0 0 0 1 1 0 0 0 0 0 0 0 0 1 1 0 1 0 0 1 0 1
 1 0 0 0 1 0 0 0 0 0 0 0 

In [12]:
predictions = m.predict([emb_matrix_test,wmal_matrix_test])

In [13]:
print(predictions)

[[1.0000000e+00 2.2970085e-08]
 [9.9982798e-01 1.7202580e-04]
 [9.9995875e-01 4.1254432e-05]
 ...
 [9.9762517e-01 2.3747978e-03]
 [9.9999917e-01 8.2827910e-07]
 [1.0000000e+00 8.8940801e-09]]


In [14]:
import numpy as np
predictions_vec = np.zeros((986,4))

In [15]:
i = 0
confidences= []
for item in predictions:
  index = np.argmax(item);
  confidences.append(item[index])
  predictions_vec[i][index] = 1
  i = i+1

In [16]:
labels_to_eval = []
for vec in predictions_vec:
  inverted = label_encoder.inverse_transform([argmax(vec)])[0]
  labels_to_eval.append(inverted)

In [17]:
confidences

[1.0,
 0.999828,
 0.99995875,
 0.9999821,
 0.73015577,
 0.9983681,
 1.0,
 0.9999995,
 1.0,
 0.9999999,
 0.99990594,
 1.0,
 0.9999598,
 0.9467617,
 0.9999504,
 0.9999999,
 1.0,
 0.999995,
 1.0,
 0.99999666,
 1.0,
 0.9611984,
 1.0,
 0.9999486,
 0.9980725,
 0.9999999,
 0.9999999,
 1.0,
 0.9999975,
 0.9934449,
 0.99999976,
 0.99999976,
 0.5700783,
 0.99999845,
 0.9714239,
 0.99999714,
 0.9999045,
 0.9963642,
 0.85809815,
 0.99998593,
 0.9314763,
 0.9987197,
 0.99999416,
 0.99999964,
 0.98327523,
 0.99999774,
 1.0,
 1.0,
 1.0,
 0.6775981,
 0.99999917,
 0.9999999,
 0.9999999,
 1.0,
 0.9999013,
 0.9999957,
 0.99999785,
 1.0,
 0.999985,
 0.99996483,
 0.99999917,
 1.0,
 0.9997334,
 0.9999542,
 0.9999908,
 0.9999174,
 0.9999988,
 0.99999917,
 0.99999976,
 1.0,
 0.99894685,
 0.99998724,
 0.9999367,
 1.0,
 1.0,
 0.78245604,
 0.996406,
 1.0,
 0.97918016,
 0.999985,
 0.9999949,
 1.0,
 0.999984,
 0.9799855,
 1.0,
 0.99821544,
 1.0,
 1.0,
 0.9999943,
 0.99999523,
 1.0,
 1.0,
 0.9997743,
 0.9999895,
 0

In [18]:
len(labels_to_eval)

986

In [19]:
dataset_test["predicted_label_neg"] = labels_to_eval
dataset_test["confidence_label_neg"] = confidences

In [20]:
!pip install scikit-learn

from sklearn.metrics import classification_report
print(classification_report(dataset_test["label_neg"],labels_to_eval,digits=5))

              precision    recall  f1-score   support

           0    0.75570   0.98028   0.85346       710
           1    0.78462   0.18478   0.29912       276

    accuracy                        0.75761       986
   macro avg    0.77016   0.58253   0.57629       986
weighted avg    0.76379   0.75761   0.69829       986



In [21]:
dataset_test

,Unnamed: 0,id,text,text_clean,Giudizio,label_pos,label_neg,label_AlBERTo_pos,label_AlBERTo_neg,label_bin_ANN,label_bin_ALB,clean,predicted_label_neg,confidence_label_neg
0,0,1,"@confagricoltura #Riso, procedura per ripristi...",procedura ripristino dazi rush finale regolame...,Negativo,1,0,0,0,10,0,<user> <hashtag> riso </hashtag> procedura per...,0,1.000000
1,1,2,"@GArancioni Gelate, xylella, sblocco Psr. A #B...",gelate xylella sblocco psr invasione video por...,Molto negativo,0,1,0,0,1,0,<user> gelate xylella sblocco psr a <hashtag> ...,0,0.999828
2,2,3,@SocialConfeuro @GArancioni #GiletArancioni. P...,confeuro protesta giusta serve coerenza assunz...,Positivo,1,1,1,0,11,10,<user> <user> <hashtag> gilet arancio ni </has...,0,0.999959
3,3,4,@_Unaitalia Uova. Timbratura all’origine neces...,uova timbratura origine necessaria tutelare co...,Negativo,1,0,0,0,10,0,<user> uova timbratura all origine necessaria ...,0,0.999982
4,4,5,@EUauditors @giamma71 / Sicurezza alimentare ...,sicurezza alimentare sostanze chimiche sistema...,Negativo,0,1,0,0,1,0,<user> <user> / sicurezza alimentare e sostanz...,1,0.730156
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
981,995,996,"My week on Twitter 🎉: 174 Mentions, 3.3M Menti...",my week on twitter mentions m mention reach li...,Neutro,0,0,0,0,0,0,my week on twitter 🎉 <number> mentions <number...,0,0.999963
982,996,997,#Tradizione torna protagonista a #tavola con a...,torna protagonista arrivo stime italiani spend...,Molto positivo,1,0,0,0,10,0,<hashtag> tradizione </hashtag> torna protagon...,0,0.999999
983,997,998,"Clima, Coldiretti: il Natale bollente con temp...",clima coldiretti natale bollente temperature a...,Negativo,0,1,0,1,1,1,clima coldiretti il natale bollente con temper...,0,0.997625
984,998,999,"USA-UE, @masgiansanti: “Possibili nuovi #dazi ...",usa ue possibili nuovi d oliva,Positivo,0,1,0,0,1,0,usa ue <user> possibili nuovi <hashtag> dazi <...,0,0.999999


In [24]:
m.save("/content/drive/MyDrive/SI_AI_KBS/integrazione_AlBERTo_WMAL/sentipolc_agritrend/models/model_1_neg_agritrend_0.57629.h5")

In [25]:
dataset_test.to_csv("/content/drive/MyDrive/SI_AI_KBS/integrazione_AlBERTo_WMAL/sentipolc_agritrend/models/predictions_AGRITREND_neg_full.csv");